# 📝 Sprint 3: Generador de Notas SOAP

**Asignatura:** Transformers del Lenguaje en Salud (UPCH)

## 🎯 Objetivo
Generar notas clínicas estructuradas en formato SOAP a partir de texto libre.

## Formato SOAP
- **S**ubjective: Síntomas reportados por el paciente
- **O**bjective: Hallazgos objetivos del examen
- **A**ssessment: Diagnóstico/evaluación del médico
- **P**lan: Plan de tratamiento

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/...)

## 1️⃣ Instalación

In [1]:
%pip install -q transformers accelerate
print("✅ Instalación completada")

Note: you may need to restart the kernel to use updated packages.
✅ Instalación completada


## 2️⃣ Cargar Modelo de Generación

In [2]:
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

# Usamos un modelo de generación de texto (ligero para Colab)
generador = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-0.5B-Instruct",  # Modelo pequeño pero capaz
    device_map="auto"
)

print("✅ Modelo cargado")

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


✅ Modelo cargado


## 3️⃣ Definir Prompt SOAP

In [3]:
def crear_prompt_soap(nota_clinica: str) -> str:
    """Crea el prompt para generar nota SOAP."""
    
    prompt = f"""Eres un asistente médico. Convierte la siguiente nota clínica en formato SOAP.

NOTA CLÍNICA:
{nota_clinica}

Genera la nota en formato SOAP:

S (Subjetivo): [Síntomas que reporta el paciente]
O (Objetivo): [Hallazgos del examen físico, signos vitales]
A (Evaluación): [Diagnóstico o impresión diagnóstica]
P (Plan): [Plan de tratamiento y seguimiento]

NOTA SOAP:
"""
    return prompt

print("✅ Función de prompt definida")

✅ Función de prompt definida


## 4️⃣ Ejemplo de Nota Clínica

In [4]:
nota_ejemplo = """
Paciente masculino de 65 años acude a consulta por dolor en el pecho 
que apareció hace 3 días. Refiere que el dolor es opresivo y empeora 
con el esfuerzo. Tiene antecedentes de hipertensión arterial y diabetes 
tipo 2. Actualmente toma losartán 50mg diario y metformina 850mg.

Al examen físico: PA 150/95 mmHg, FC 88 lpm, FR 18 rpm, SpO2 96%.
Auscultación cardíaca con ruidos rítmicos, sin soplos. Pulmones claros.
ECG muestra ritmo sinusal con alteraciones inespecíficas de ST.
"""

print("📋 Nota clínica de ejemplo cargada")
print(nota_ejemplo)

📋 Nota clínica de ejemplo cargada

Paciente masculino de 65 años acude a consulta por dolor en el pecho 
que apareció hace 3 días. Refiere que el dolor es opresivo y empeora 
con el esfuerzo. Tiene antecedentes de hipertensión arterial y diabetes 
tipo 2. Actualmente toma losartán 50mg diario y metformina 850mg.

Al examen físico: PA 150/95 mmHg, FC 88 lpm, FR 18 rpm, SpO2 96%.
Auscultación cardíaca con ruidos rítmicos, sin soplos. Pulmones claros.
ECG muestra ritmo sinusal con alteraciones inespecíficas de ST.



## 5️⃣ Generar Nota SOAP

In [5]:
def generar_soap(nota_clinica: str) -> str:
    """Genera nota SOAP a partir de texto clínico."""
    
    prompt = crear_prompt_soap(nota_clinica)
    
    resultado = generador(
        prompt,
        max_new_tokens=400,
        temperature=0.3,
        do_sample=True,
        pad_token_id=generador.tokenizer.eos_token_id
    )
    
    # Extraer solo la parte generada
    texto_completo = resultado[0]['generated_text']
    nota_soap = texto_completo.split("NOTA SOAP:")[-1].strip()
    
    return nota_soap

print("🔄 Generando nota SOAP...\n")
soap = generar_soap(nota_ejemplo)

print("="*60)
print("📝 NOTA SOAP GENERADA")
print("="*60)
print(soap)

🔄 Generando nota SOAP...

📝 NOTA SOAP GENERADA
El paciente presenta dolor en el pecho que ha sido observado durante los últimos 3 días. El dolor es opresivo y se mantiene persistente al realizar actividades físicas. Se refiere a una hipertensión arterial y diabetes tipo 2, y actualmente está utilizando losartán 50 mg diariamente y metformina 850 mg. Su estado general es estable y no presenta síntomas de fiebre, náuseas o diarrea. El paciente tiene un rango de oxígeno de 96%, lo cual indica que su sangre está suficientemente pura para el funcionamiento de las células. La auscultación cardíaca presenta ritmo sinus sinusal, sin soplo. Sin embargo, el paciente presenta un ruido pulmonar, lo que sugiere que podría haber un problema pulmonar. En cuanto a los signos vitales, el paciente tiene un rango de oxígeno de 150/95 mmHg, lo que indica un nivel de oxígeno bajo. Los signos vitales son consistentes con un estado de salud crónico. Por lo tanto, se puede inferir que el paciente presenta un 

## 6️⃣ Función de Auto-Auditoría

In [6]:
def auditar_soap(nota_soap: str) -> dict:
    """Verifica que la nota SOAP tenga todas las secciones."""
    
    secciones = {
        "S (Subjetivo)": "S" in nota_soap or "Subjetivo" in nota_soap,
        "O (Objetivo)": "O" in nota_soap or "Objetivo" in nota_soap,
        "A (Evaluación)": "A" in nota_soap or "Evaluación" in nota_soap or "Assessment" in nota_soap,
        "P (Plan)": "P" in nota_soap or "Plan" in nota_soap
    }
    
    completa = all(secciones.values())
    
    return {
        "secciones": secciones,
        "completa": completa,
        "porcentaje": sum(secciones.values()) / len(secciones) * 100
    }

# Auditar la nota generada
auditoria = auditar_soap(soap)

print("\n🔍 AUDITORÍA DE LA NOTA")
print("="*40)
for seccion, presente in auditoria['secciones'].items():
    icono = "✅" if presente else "❌"
    print(f"{icono} {seccion}")

print(f"\n📊 Completitud: {auditoria['porcentaje']:.0f}%")


🔍 AUDITORÍA DE LA NOTA
✅ S (Subjetivo)
✅ O (Objetivo)
✅ A (Evaluación)
✅ P (Plan)

📊 Completitud: 100%


## 7️⃣ Prueba con tu propia nota

In [7]:
# 👇 Escribe aquí tu nota clínica
mi_nota = """
Paciente femenina de 45 años con dolor abdominal de 2 días de evolución.
Refiere náuseas y vómitos. Sin fiebre. Al examen: abdomen blando, 
dolor en hipocondrio derecho, Murphy positivo.
"""

print("🔄 Procesando tu nota...\n")
mi_soap = generar_soap(mi_nota)

print("📝 TU NOTA SOAP:")
print("="*60)
print(mi_soap)

# Auditar
mi_auditoria = auditar_soap(mi_soap)
print(f"\n📊 Completitud: {mi_auditoria['porcentaje']:.0f}%")

🔄 Procesando tu nota...

📝 TU NOTA SOAP:
El paciente presenta síntomas de dolor abdominal de 2 días, náuseas y vómitos, sin fiebre. Se observa un abdomen blando, dolor en el hipocondrio derecho y Murphy positivo. No se ha detectado fiebre o fiebre alta. El paciente está en estado de alerta y requiere atención médica inmediata. 

S (Subjectivo): El paciente presenta síntomas de dolor abdominal de 2 días, náuseas y vómitos, sin fiebre. Se observa un abdomen blando, dolor en el hipocondrio derecho y Murphy positivo.
O (Objective): El paciente es una mujer de 45 años con dolor abdominal de 2 días de evolución.
A (Evaluación): El paciente presenta síntomas de dolor abdominal de 2 días, náuseas y vómitos, sin fiebre. Se observa un abdomen blando, dolor en el hipocondrio derecho y Murphy positivo.
P (Plan): Inmediatamente se procederá a la evaluación y tratamiento para identificar los causas de estos síntomas y establecer una estrategia de tratamiento adecuada. Una vez identificados los probl

## ⚠️ Consideraciones Éticas

- Las notas generadas por IA **deben ser revisadas** por el médico
- No reemplazan el juicio clínico profesional
- Los datos sensibles deben manejarse con confidencialidad
- Este es un sistema **experimental** para fines educativos